In [1]:
!git clone https://github.com/ieee8023/covid-chestxray-dataset.git

Cloning into 'covid-chestxray-dataset'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 1741 (delta 69), reused 84 (delta 55), pack-reused 1616
Receiving objects: 100% (1741/1741), 221.08 MiB | 36.88 MiB/s, done.
Resolving deltas: 100% (843/843), done.
Checking out files: 100% (357/357), done.


In [0]:
import torch
import torchvision
from PIL import Image
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torch import nn,optim
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [0]:
model=torchvision.models.googlenet(pretrained=True)

In [0]:
model.fc=nn.Sequential(nn.Linear(1024,1),
                       nn.Sigmoid())

In [22]:
model.load_state_dict(torch.load('best_model.pt',map_location=torch.device('cpu')))

<All keys matched successfully>

For predicting a new image we need to pass the location or path of the image and model to the __predict__ function below

In [0]:
def predict(image_dir,model,logits=False):
  image = Image.open(image_dir).convert('RGB')
  image = np.array(image.resize((256,256)))
  image = image[:,:,0]
  image = np.uint8(((np.array(image)/255).reshape(256,256,1))*255*255)
  image = np.tile(image,3)
  transform = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Resize((224,224)),
                                  transforms.ToTensor(),
                                  transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
                                  ])
  image=transform(image)
  image=image.unsqueeze(0)
  if torch.cuda.is_available():
    model.cuda()
    image=image.cuda()
  with torch.no_grad():
    output=model(image)
  if logits==True:
    return output.detach().cpu().view(-1).numpy()
  else:
    return torch.round(output).detach().cpu().view(-1).numpy()

In [21]:
image_path='/content/covid-chestxray-dataset/images/01E392EE-69F9-4E33-BFCE-E5C968654078.jpeg'
print(predict(image_path,model))

[1.]
